In [1]:
#!/usr/bin/env python3


import argparse
import csv
import os
from requests import Session
from typing import Generator, TypeVar
import requests

S2_API_KEY = "mEwrkvDmRc9KFXu76cAAk16KJxHNNUuf9tYaot1g"


def get_paper_batch( ids, fields):
    params = {
        'fields': fields
    }
    headers = {
        'X-API-KEY': S2_API_KEY,
    }
    body = {
        'ids': ids,
    }

    # https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/post_graph_get_papers
    with requests.post('https://api.semanticscholar.org/graph/v1/paper/batch',
                       params=params,
                       headers=headers,
                       json=body
                       ) as response:
        return response.content


print(get_paper_batch(["e28bdc373de80d7ec0e64631a89e64fbdcdae230"],["citations"]))

b'[{"paperId": "e28bdc373de80d7ec0e64631a89e64fbdcdae230", "citations": [{"paperId": "874b5d24ee73541b530e1498d2af7ab081adac65", "title": "The influence of the city metaphor and its derivates in software visualization"}, {"paperId": "9be328947a66ff6940ab71814910008bf2df3fe8", "title": "Apples, Oranges, and Software Engineering: Study Selection Challenges for Secondary Research on Latent Variables"}, {"paperId": "e3622615d6f32c1c47d6fa9aae583d08163eb9ca", "title": "Railway Catenary Condition Monitoring: A Systematic Mapping of Recent Research"}, {"paperId": "3bf635f159bc682117f2cd34dcd8cb7a6a6715a8", "title": "Informing the State of Process Modeling and Automation of Blood Banking and Transfusion Services Through a Systematic Mapping Study"}, {"paperId": "0259549394e4a9a1a565b28082c15a8ad0376013", "title": "Agile human resource management: A systematic mapping study"}, {"paperId": "86ba1cd9010e70c745a7ce354b26b81a40ab3515", "title": "Fintech research: systematic mapping, classification,

In [4]:
import json
import codecs
# with open('s2orc_20mb.txt', 'r', encoding='utf-8') as f:
#     jsonObject = json.load(f)
# f.close()
with codecs.open("s2orc_4000.json", 'r', encoding='utf-16') as fdata:
    count = 0
    results_json = []

    def getIntroduction(lineJson):
        sectionheaders = json.loads(lineJson["content"]["annotations"]["sectionheader"])
        # print(sectionheaders)
        start_index = 0
        end_index = 0
        second = False
        for i in sectionheaders:
            if "attributes" in i and start_index == 0:
                start_index = i["start"]
                second = True
            elif "attributes" in i and second:
                end_index = i["end"]
                break
            # print(corpus[i["start"]:i["end"]])
        #     break
        paragraphs = json.loads(lineJson["content"]["annotations"]["paragraph"])
        corpus = lineJson["content"]["text"]
        # print(start_index,end_index)
        introduction_para = []
        for index,i in enumerate(paragraphs):
            if i["start"] >= start_index and i["end"] <= end_index:
                introduction_para.append(corpus[i["start"]:i["end"]])
                # print(corpus[i["start"]:i["end"]])
            # print(str(index)+": ")
            # print(corpus[i["start"]:i["end"]])
        return "\n".join(introduction_para)
        
    while True:
        count += 1
        line = fdata.readline()
        if len(line)==0:
            break
        # print(line)
        try:
            lineJson = json.loads(line)
            results_json.append(lineJson)
            # introduction = getIntroduction(lineJson)
            corpus = lineJson["content"]["text"].lower()
            if "research question " in corpus:
                rq_tmp = corpus.split("research question ")[1].split(".")[0]
                print(rq_tmp)
            # break

        except Exception as e:
            print(e)
            print("failed: "+str(count))
            # break

# paperIdList = [i for i in id2introduction]
# for i in lineJson["content"]["annotations"]:
#     print(i)
# print(json.loads(lineJson["content"]["annotations"]["paragraph"]))

was developed: why are men not involved in various aspects of reproductive health care?


search strategy

the databases, including pubmed, scopus, web of science, cochrane, and proquest, were searched, systematically
guiding this review was: what are the barriers and facilitators of the uptake of dht in cardiovascular care?



numerous nlp tasks are knowledge-intensive: they require recalling and synthesizing facts from a knowledge source (e
'NoneType' object has no attribute 'lower'
failed: 157
therefore asked: what participatory methods exist that involve health workers themselves in identifying challenges and potential solutions at work, and to what extent have they generated positive changes?


methodology

the protocol for this systematic review was registered with the international prospective register of systematic reviews (pros-pero, crd42021261050) in june 2021
'NoneType' object has no attribute 'lower'
failed: 252
'NoneType' object has no attribute 'lower'
failed: 253
of the

In [3]:
lineJson["content"]["annotations"]["sectionheader"]

'[{"end":2318,"start":2293},{"end":2818,"start":2810},{"end":3169,"start":3161},{"end":3227,"start":3219}]'

In [ ]:
count = 0
paperJsonSampled = {}
for i in paperJson:
    if count <50000:
        if paperJson[i]["abstract"] is not None and len(paperJson[i]["authors"])>=2 and paperJson[i]["introduction"] is not None and paperJson[i]["title"] is not None:
            paperJsonSampled[i] = paperJson[i]
            count = count +1
    else:
        break
print(len(paperJsonSampled))

with open('data_set_v2.0/paper_json.json', 'w', encoding='utf-8') as f:
    json.dump(paperJsonSampled, f, indent=4)

author_dict = {}
authorCount = 0
for paper in paperJsonSampled:
    authorCount = authorCount + len(paperJsonSampled[paper]["authors"])
    for author in paperJsonSampled[paper]["authors"]:
        # print(author)
        author_dict[author["authorId"]]=author["name"]

author_json = {}
print(authorCount/len(paperJsonSampled))
len(author_dict)

# print(paperJson[0])

In [ ]:

import time
author_id_list = []
for index, i in enumerate(author_dict):
    if index >=0:
        author_json[i] = {"id":i, "name":author_dict[i], "papers":[]}
        if index % 4 == 0:
            print(str(index)+" started")
            author_id_list_real = []
            for j in author_id_list:
                if j is not None:
                    author_id_list_real.append(j)
            if len(author_id_list_real)>0:
                tmp_results = sch.get_authors(author_id_list_real)
                for author_tmp in tmp_results:
                    papers = author_tmp.papers
                    author_json[author_tmp.authorId]["paperCount"] = author_tmp["paperCount"]
                    author_json[author_tmp.authorId]["citationCount"] = author_tmp["citationCount"]
                    author_json[author_tmp.authorId]["hIndex"] = author_tmp["hIndex"]
                    author_json[author_tmp.authorId]["homepage"] = author_tmp["homepage"]
                    # print(papers)
                    for paper in papers:
                        if paper.abstract !=None:
                            author_json[author_tmp.authorId]["papers"].append({"title":paper.title, "abstract":paper.abstract,"year":paper.year,"fieldsOfStudy":paper.fieldsOfStudy})
                author_id_list = []
                print(str(index)+" done!")
                time.sleep(2)
                
                
        else:
            author_id_list.append(i)
# for i in results[0]:
#     print(i)
#     # print("\t",results[0][i])
# len(results[0].papers)
# author_json

In [ ]:
author_dict = {}
author_json_filtered = {}
for i in author_json:
    if len(author_json[i]["papers"])>0:
        author_dict[i] = author_json[i] 
        author_json_filtered[i]= author_json[i] 
        # print(author_json[i])
        # break
len(author_dict)
paperJsonFiltered = {}
for i in paperJsonSampled:
    flag = True
    for author in paperJson[i]["authors"]:
        if author["authorId"] not in author_dict:
            flag= False
            break
    if flag:
        paperJsonFiltered[i] = paperJson[i]
print(len(paperJsonFiltered))
# with open('data_set_v2.0/author_json.json', 'w', encoding='utf-8') as f:
#     json.dump(author_json, f, indent=4)
with open('data_set_v2.1/author_json.json', 'w', encoding='utf-8') as f:
    json.dump(author_json_filtered, f, indent=4)
with open('data_set_v2.1/paper_json.json', 'w', encoding='utf-8') as f:
    json.dump(paperJsonFiltered, f, indent=4)

In [13]:
import json
with open('data_set_v2.1/paper_json.json', 'r',encoding='utf-8') as f:
    paperJson = json.load(f)
print(len(paperJson))
with open('data_set_v2.1/author_json.json', 'r',encoding='utf-8') as f:
    authorJson = json.load(f)
print(len(authorJson))

605
3535


In [5]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('data_set_v3.0/author_json.json', 'r',encoding='utf-8') as f:
    authors_sampled = json.load(f)
print(len(authors_sampled))

with open('data_set_v3.0/paper_json.json', 'r',encoding='utf-8') as f:
    papers_sampled = json.load(f)
print(len(papers_sampled))
# sampled papers to 500 
# count = 0
# max_paper = 100
# author_paper_count = []
# paper_author_count = []
# papers_sampled = {}
# authors_sampled = {}
# for i in papers:
#     if count < 100:
#         flag = True
#         for author in papers[i]["authors"]:
#             if len(authors[author]["papers"])<5:
#                 flag = False
#         if flag and len(papers[i]["authors"]) > 1:
#             paper_author_count.append(len(papers[i]["authors"]))
#             papers_sampled[i] = papers[i]
#             papers_sampled[i]["author_history"] = {}
            
#             for author in papers[i]["authors"]:
#                 papers_sampled[i]["author_history"][author] = {}
#                 papers_sampled[i]["author_history"][author]["authors_history_text"] = [papers[i]["abstract"]]
#                 papers_sampled[i]["author_history"][author]["authors_history_dict"] = [{"Abstract":"","Title":""}]
#                 paper_count_tmp = 0
#                 for paper_tmp in authors[author]["papers"]:
#                     if paper_count_tmp < max_paper and paper_tmp["title"] != papers[i]["title"]:
#                         papers_sampled[i]["author_history"][author]["authors_history_text"].append("Abstract: "+paper_tmp["abstract"]+"\nTitle: "+paper_tmp["title"])
#                         papers_sampled[i]["author_history"][author]["authors_history_dict"].append({"Abstract":paper_tmp["abstract"],"Title":paper_tmp["title"]})
#                         paper_count_tmp = paper_count_tmp + 1
#                 author_paper_count.append(len(authors[author]["papers"]))
#                 authors_sampled[author] = authors[author]
#             count = count + 1
# print("papers count after sampled: ",len(papers_sampled))
# print("authors count after sampled: ",len(authors_sampled))
# print(len(authorJson))
# for i in paperJsonSampled:
#     print(paperJsonSampled[i])
#     break
# paperDictId = {i:paperJson[i]["year"] for i in paperJson}
# paperDictIdSorted = {k: v for k, v in sorted(paperDictId.items(), key=lambda item: item[1], reverse=True)}
# count = 0


# print(len(paperJsonSampled),len(authors_sampled))
from scholarly import scholarly

with open("naive_method_v3.0/author_interests.tsv", "wb", buffering=0) as out_file:
    for index,authorId in enumerate(authors_sampled):
        try:
            search_query = scholarly.search_author(authors_sampled[authorId]["name"])
            first_author_result = next(search_query)
            print(str(index)+" : "+str(first_author_result["interests"]))
        except Exception as e:
            print(str(index)+" "+authors_sampled[authorId]["name"]+" failed")
            continue
        write_str = bytes(str(authorId)+"\t"+str(first_author_result["interests"])+"\n", 'utf-8')
        out_file.write(write_str)

281
107
0 : []
1 : ['Software Engineering', 'Empirical Software Engineering', 'SBSE', 'Behavioral Software Engineering', 'Artificial Intelligence']
2 : ['Livestock Production', 'Animal Nutrition', 'Animal Health and Production']
3 : ['Software engineering']
4 : ['Large Language Models', 'Data Science', 'Software Engineering', 'Blockchain', 'Smart Contracts']
5 : ['Software Engineering', 'Agile Methodologies', 'Software Metrics', 'Econophysics', 'Blockchain and cryptocurrencies']
6 : ['Natural Language Processing', 'Machine Learning', 'Human Aspect in Software Engineering']
7 : ['Respiratory Disease']
8 : ['Formal modelling of interactive and complex …']
9 : ['Model based testing', 'distributed testing', 'mutation testing', 'formal methods', 'SBSE']
10 : ['Open Source', 'Inner Source', 'Crowdsourcing', 'Software Engineering', 'Software Ecosystems']
11 : ['Agile methods', 'open source software', 'information systems development', 'inner_source', 'DevOps']
12 : ['Physiology', 'Vascular Bi

In [28]:
from scholarly import scholarly

for index,authorId in enumerate(authors_sampled):
    try:
        search_query = scholarly.search_author(authors_sampled[authorId]["name"])
        first_author_result = next(search_query)
        authors_sampled[authorId]["interests"] = first_author_result["interests"]
        print(str(index)+" : "+str(first_author_result["interests"]))
    except Exception as e:
        print(str(index)+" "+authors_sampled[authorId]["name"]+" failed")
        continue